In [11]:
# === CELDA DE CONEXIÓN (ejecutar una sola vez) ===
import comtypes.client

# Conectar a instancia activa de SAP2000
try:
    SapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    SapModel = SapObject.SapModel
    print("✅ Conectado a SAP2000")
    print(f"📁 Archivo: {SapModel.GetModelFilename()}")
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("   Asegúrate de que SAP2000 esté abierto.")

✅ Conectado a SAP2000
📁 Archivo: (Untitled)


In [14]:
def create_mesh_by_coord(SapModel, width, length, nx, ny, start_x=0.0, start_y=0.0, start_z=0.0, plane="XY", prop_name="Default"):
    """
    Crea una malla rectangular de áreas usando AddByCoord en el plano especificado.
    
    Args:
        SapModel: Objeto de conexión a SAP2000.
        width (float): Dimensión 1 (X en XY/XZ, Y en YZ).
        length (float): Dimensión 2 (Y en XY, Z en XZ/YZ).
        nx (int): Número de divisiones en Dimensión 1.
        ny (int): Número de divisiones en Dimensión 2.
        start_x, start_y, start_z (float): Coordenadas de la esquina origen.
        plane (str): Plano de dibujo ("XY", "XZ", "YZ").
        prop_name (str): Nombre de la propiedad de área a asignar.
        
    Returns:
        list: Lista de nombres de las áreas creadas.
    """
    created_areas = []
    
    d1 = width / nx
    d2 = length / ny
    
    print(f"Generando malla {nx}x{ny} en plano {plane} (d1={d1:.2f}, d2={d2:.2f})...")
    
    for i in range(nx):
        for j in range(ny):
            # Coordenadas locales 2D (u, v)
            u0 = i * d1
            v0 = j * d2
            
            # 4 esquinas en local 2D (antihorario)
            # 1: (u0, v0), 2: (u0+d1, v0), 3: (u0+d1, v0+d2), 4: (u0, v0+d2)
            us = [u0, u0 + d1, u0 + d1, u0]
            vs = [v0, v0, v0 + d2, v0 + d2]
            
            xs, ys, zs = [], [], []
            
            for k in range(4):
                u, v = us[k], vs[k]
                if plane.upper() == "XY":
                    xs.append(start_x + u)
                    ys.append(start_y + v)
                    zs.append(start_z)
                elif plane.upper() == "XZ":
                    xs.append(start_x + u)
                    ys.append(start_y)
                    zs.append(start_z + v)
                elif plane.upper() == "YZ":
                    xs.append(start_x)
                    ys.append(start_y + u)
                    zs.append(start_z + v)
            
            try:
                # AddByCoord(NumberPoints, x, y, z, Name, PropName, UserName, CSys)
                ret = SapModel.AreaObj.AddByCoord(4, xs, ys, zs, "", prop_name, "", "Global")
                
                if isinstance(ret, (list, tuple)) and ret[-1] == 0:
                    created_areas.append(str(ret[0]))
                elif isinstance(ret, int) and ret == 0:
                    pass
                else:
                    print(f"Error creando área en celda ({i},{j}): {ret}")
                    
            except Exception as e:
                print(f"Excepción en celda ({i},{j}): {e}")
                
    print(f"Se crearon {len(created_areas)} áreas en {plane}.")
    return created_areas

In [18]:
# === PRUEBA DE GENERACIÓN DE MALLA ===

# Parámetros de prueba
test_width = 500.0   # mm
test_length = 500.0  # mm
test_nx = 5          # 5 divisiones
test_ny = 5          # 5 divisiones

# Prueba 1: Plano XY
print("--- Prueba XY ---")
create_mesh_by_coord(
    SapModel, test_width, test_length, test_nx, test_ny, 
    start_x=1000.0, start_y=0.0, start_z=0.0, 
    plane="XY", prop_name="Default"
)

# Parámetros de prueba
test_width = 1000.0   # mm
test_length = 700.0  # mm
test_nx = 3          # 3 divisiones
test_ny = 5          # 5 divisiones

# Prueba 2: Plano XZ (Pared vertical)
print("\n--- Prueba XZ ---")
create_mesh_by_coord(
    SapModel, test_width, test_length, test_nx, test_ny, 
    start_x=2000.0, start_y=0.0, start_z=0.0, 
    plane="XZ", prop_name="Default"
)

# Parámetros de prueba
test_width = 3000.0   # mm
test_length = 700.0  # mm
test_nx = 7          # 7 divisiones
test_ny = 5          # 5 divisiones

# Prueba 3: Plano YZ (Pared lateral)
print("\n--- Prueba YZ ---")
create_mesh_by_coord(
    SapModel, test_width, test_length, test_nx, test_ny, 
    start_x=500.0, start_y=0.0, start_z=0.0, 
    plane="YZ", prop_name="Default"
)

# Refrescar vista
SapModel.View.RefreshView(0, False)

--- Prueba XY ---
Generando malla 5x5 en plano XY (d1=100.00, d2=100.00)...
Se crearon 25 áreas en XY.

--- Prueba XZ ---
Generando malla 3x5 en plano XZ (d1=333.33, d2=140.00)...
Se crearon 15 áreas en XZ.

--- Prueba YZ ---
Generando malla 7x5 en plano YZ (d1=428.57, d2=140.00)...
Se crearon 35 áreas en YZ.


0